In [1]:
import torch
import torch.nn as nn
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
from scipy.linalg import sqrtm
import os


from google.colab import drive
drive.mount('/content/gdrive')


if torch.cuda.is_available():
  print("hello GPU")
else:
  print("sadge")


class BaseModel(nn.Module, ABC):
    def __init__(self):
        super().__init__()

    @abstractmethod
    def forward(self, x, interm=False):
        pass

    @property
    def device(self):
        return next(self.parameters()).device

    def restore_checkpoint(self, ckpt_file, optimizer=None):
        if not ckpt_file:
            raise ValueError("No checkpoint file to be restored.")

        try:
            ckpt_dict = torch.load(ckpt_file)
        except RuntimeError:
            ckpt_dict = torch.load(ckpt_file,
                                   map_location=lambda storage, loc: storage)

        # Restore model weights
        self.load_state_dict(ckpt_dict['model_state_dict'])

        # Restore optimizer status if existing. Evaluation doesn't need this
        if optimizer:
            optimizer.load_state_dict(ckpt_dict['optimizer_state_dict'])

        # Return global step
        return ckpt_dict['global_step']

    def save_checkpoint(self,
                        directory,
                        global_step,
                        optimizer=None,
                        name=None):
        # Create directory to save to
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Build checkpoint dict to save.
        ckpt_dict = {
            'model_state_dict': self.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer is not None else None,
            'global_step': global_step
        }

        # Save the file with specific name
        if name is None:
            name = "{}_{}_steps.pth".format(
                os.path.basename(directory),  # netD or netG
                global_step)

        torch.save(ckpt_dict, os.path.join(directory, name))

    def count_params(self):
        num_total_params = sum(p.numel() for p in self.parameters())
        num_trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)

        return num_total_params, num_trainable_params


class DBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size=3,
                 stride=1,
                 padding=0,
                 downsample=False):
        super().__init__()
        self.downsample = downsample
        self.c1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.activation = nn.LeakyReLU(0.2)

        nn.init.normal_(self.c1.weight.data, 0.0, 0.02)

    def forward(self, x):
        h = x
        h = self.c1(h)
        h = self.activation(h)
        if self.downsample:
            h = F.avg_pool1d(h, 2)
        return h

class Discriminator(BaseModel):
    def __init__(self, **kwargs):
        super().__init__()
        self.count = 0
        self.errD_array = []
        self.bce = nn.BCELoss()
        self.ndf = 100

        self.sblock1 = DBlock(64, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock2 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.sblock3 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock4 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.sblock5 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock6 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.sblock7 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock8 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.sblock9 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock10 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.sblock11 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=False)
        self.sblock12 = DBlock(self.ndf, self.ndf, kernel_size=9, stride=1, padding=0, downsample=True)
        self.c = nn.Conv1d(self.ndf, 64, 1, 1, 0)
        self.end = nn.Linear(33, 1)
        self.sigmoid = nn.Sigmoid()

        nn.init.normal_(self.c.weight.data, 0.0, 0.02)
        nn.init.normal_(self.end.weight.data, 0.0, 0.02)

    def forward(self, x, interm=False):
        x = x.float()
        h = self.sblock1(x)
        h = self.sblock2(h)
        h = self.sblock3(h)
        h = self.sblock4(h)
        h = self.sblock5(h)
        h = self.sblock6(h)
        h = self.sblock7(h)
        h = self.sblock8(h)
        h = self.sblock9(h)
        h = self.sblock10(h)
        h = self.sblock11(h)
        h = self.sblock12(h)
        h = self.c(h)

        interm_out = self.end(h)
        if interm:
            return interm_out
        h = interm_out

        h = self.sigmoid(h)
        return h.view(h.shape[0], 64)

    def compute_loss(self, output, actual):
        return self.bce(output, actual)

def train(open, closed, dupe=False):
  device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
  netD = Discriminator().to(device)
  optimizer = torch.optim.Adam(netD.parameters(), 0.0001, (0.5, 0.99))

  train_lossD = []
  train_accuracyD = []
  test_lossD = []
  test_accuracyD = []

  # Split data into training and testing sets
  train_open, test_open = train_test_split(open, test_size=0.2, random_state=42)
  train_closed, test_closed = train_test_split(closed, test_size=0.2, random_state=42)

  if (dupe):
      train_open = np.concatenate((train_open, train_open))
      train_closed = np.concatenate((train_closed, train_closed))

  # Move data to tensors and to device
  train_open = torch.tensor(train_open).float().to(device)
  train_closed = torch.tensor(train_closed).float().to(device)
  test_open = torch.tensor(test_open).float().to(device)
  test_closed = torch.tensor(test_closed).float().to(device)

  for epoch in range(100):
    # Training
    netD.zero_grad()

    train_output_open = netD.forward(train_open)
    train_output_closed = netD.forward(train_closed)

    train_vals_open = torch.full(train_output_open.shape, 1.0, dtype=torch.float, device=device)
    train_vals_closed = torch.full(train_output_closed.shape, 0.0, dtype=torch.float, device=device)

    train_pred_labels = torch.cat((train_output_open, train_output_closed))
    train_actual_labels = torch.cat((train_vals_open, train_vals_closed))

    train_accuracy = ((train_pred_labels > 0.5) == train_actual_labels).float().mean()
    train_accuracyD.append(train_accuracy.item())

    # Compute training loss
    train_errD_real = netD.compute_loss(train_output_open, train_vals_open)
    train_errD_fake = netD.compute_loss(train_output_closed, train_vals_closed)
    train_errD = train_errD_real + train_errD_fake
    train_errD.backward()
    optimizer.step()

    train_lossD.append(train_errD.item())

    # Testing
    with torch.no_grad():
      test_output_open = netD.forward(test_open)
      test_output_closed = netD.forward(test_closed)

      # Compute classification accuracy
      test_vals_open = torch.full(test_output_open.shape, 1.0, dtype=torch.float, device=device)
      test_vals_closed = torch.full(test_output_closed.shape, 0.0, dtype=torch.float, device=device)

      test_pred_labels = torch.cat((test_output_open, test_output_closed))
      test_actual_labels = torch.cat((test_vals_open, test_vals_closed))

      test_accuracy = ((test_pred_labels > 0.5) == test_actual_labels).float().mean()

      # Bin guess into T/F for confusion matrix
      test_output_open_cm = (test_output_open.mean(dim=1, keepdim=True) > 0.5).float()
      test_output_closed_cm = (test_output_closed.mean(dim=1, keepdim=True) > 0.5).float()
      test_vals_open_cm = (test_vals_open.mean(dim=1, keepdim=True) > 0.5).float()
      test_vals_closed_cm = (test_vals_closed.mean(dim=1, keepdim=True) > 0.5).float()

      # Compute testing loss
      test_errD_real = netD.compute_loss(test_output_open, test_vals_open)
      test_errD_fake = netD.compute_loss(test_output_closed, test_vals_closed)
      test_errD = test_errD_real + test_errD_fake

      test_accuracyD.append(test_accuracy.item())
      test_lossD.append(test_errD.item())

    # if epoch % 20 == 0:
    #   print('Epoch', epoch)
    #   print('Train Loss:', train_errD.item(), 'Test Loss:', test_errD.item())
    #   print('Train Accuracy:', train_accuracy.item(), 'Test Accuracy:', test_accuracy.item())
    #   plt.figure(figsize=(10,4))
    #   plt.subplot(1, 2, 1)
    #   plt.plot(train_lossD, label='Training Loss')
    #   plt.plot(test_lossD, label='Testing Loss')
    #   plt.legend()
    #   plt.subplot(1, 2, 2)
    #   plt.plot(train_accuracyD, label='Training Accuracy')
    #   plt.plot(test_accuracyD, label='Testing Accuracy')
    #   plt.legend()
    #   plt.show()

  print('Final Test Accuracy:', test_accuracy.item())
  return torch.cat((test_output_open_cm, test_output_closed_cm)), torch.cat((test_vals_open_cm, test_vals_closed_cm)), netD, test_accuracy.item()




Mounted at /content/gdrive
hello GPU


In [8]:
import numpy as np
import scipy
from scipy.linalg import sqrtm
from scipy.linalg import sqrtm
from scipy import linalg
import numpy as np

open = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-open-64ch.npy")
closed = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-closed-64ch.npy")
open_generated = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-open-1.npy")
closed_generated = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-closed-1.npy")

# def calculate_FID(real, fake):
#     real_data = real.float().to('cuda')
#     generated_data = fake.float().to('cuda')

#     with torch.no_grad():
#         act_real = netD(real_data, interm=True)
#         act_generated = netD(generated_data, interm=True)

#     # Reshaping the activations and treating each time-series data point as an individual instance
#     act_real = act_real.reshape(-1, act_real.shape[-1])
#     act_generated = act_generated.reshape(-1, act_generated.shape[-1])

#     act_real = act_real.cpu().numpy()
#     act_generated = act_generated.cpu().numpy()

#     # Calculate mean and covariance statistics
#     mu_real, sigma_real = act_real.mean(axis=0), np.cov(act_real, rowvar=False)
#     mu_gen, sigma_gen = act_generated.mean(axis=0), np.cov(act_generated, rowvar=False)

#     # Calculate sum squared difference between means
#     ssdiff = np.sum((mu_real - mu_gen) ** 2.0)

#     # Calculate sqrt of product between cov
#     covmean, _ = scipy.linalg.sqrtm(sigma_real.dot(sigma_gen), disp=False)

#     # Check and correct imaginary numbers from sqrt
#     if np.iscomplexobj(covmean):
#         covmean = covmean.real

#     # Sum of the differences in covariance
#     fid = ssdiff + np.trace(sigma_real + sigma_gen - 2.0 * covmean)
#     # fid = fid / 10000
#     print('FID Score: %.3f' % fid)



def calculate_FID(real, fake, netD):
    real_data = real.float().to('cuda')
    generated_data = fake.float().to('cuda')

    with torch.no_grad():
        act_real = netD(real_data, interm=True)
        act_generated = netD(generated_data, interm=True)

    # Reshaping the activations and treating each time-series data point as an individual instance
    act_real = act_real.reshape(-1, act_real.shape[-1])
    act_generated = act_generated.reshape(-1, act_generated.shape[-1])

    act_real = act_real.cpu().numpy()
    act_generated = act_generated.cpu().numpy()

    # Calculate mean and covariance statistics
    mu_real, sigma_real = act_real.mean(axis=0), np.cov(act_real, rowvar=False)
    mu_gen, sigma_gen = act_generated.mean(axis=0), np.cov(act_generated, rowvar=False)

    fid = calculate_frechet_distance(mu_real, sigma_real, mu_gen, sigma_gen)
    return fid

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    if mu1.shape != mu2.shape or sigma1.shape != sigma2.shape:
        raise ValueError(
            "(mu1, sigma1) should have exactly the same shape as (mu2, sigma2)."
        )

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        print(
            "WARNING: fid calculation produces singular product; adding {} to diagonal of cov estimates"
            .format(eps))

        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D
# from tensorflow.keras import backend as K
# import tensorflow as tf
# import numpy as np
# from scipy.linalg import sqrtm
# from scipy import linalg
# import numpy as np


# def calculate_fidt(signals1, signals2, feature_extractor):

#   # Convert Tensors to CUDA
#   signals1 = signals1.cuda()
#   signals2 = signals2.cuda()

#   with torch.no_grad():
#       act1 = feature_extractor(signals1, interm=True).reshape((signals1.shape[0], signals1.shape[1]))
#       act2 = feature_extractor(signals2, interm=True).reshape((signals2.shape[0], signals2.shape[1]))

#   act1 = act1.cpu().numpy()
#   act2 = act2.cpu().numpy()

#   # Calculate mean and covariance
#   mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
#   mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)

#   # Calculate sum of squares difference between means
#   ssdiff = np.sum((mu1 - mu2)**2.0)

#   # Calculate sqrt of product between cov
#   covmean = sqrtm(sigma1.dot(sigma2))

#   # Check and correct imaginary numbers from sqrt
#   if np.iscomplexobj(covmean):
#       covmean = covmean.real

#   # Calculate score
#   fid = ssdiff + np.trace(np.absolute(sigma1 + sigma2 - 2.0 * covmean))  # modified line
#   return fid


# Calculate FID scores

average 100 iterations

In [9]:
open_realfake = []
closed_realfake = []
open_realreal = []
closed_realreal = []
open_realnoise = []
closed_realnoise = []


for _ in range(100):
  predicted_labels_synthetic_classifier, true_labels_synthetic_classifier, netD, accuracy = train(open, closed)

  # open_realfake
  real_data = torch.tensor(open).float()
  generated_data = torch.tensor(open_generated).float()
  open_realfake.append(calculate_FID(real_data, generated_data, netD)/10)

  # closed_realfake
  real_data = torch.tensor(closed).float()
  generated_data = torch.tensor(closed_generated).float()
  closed_realfake.append(calculate_FID(real_data, generated_data, netD))

  # open_realreal
  real_data = torch.tensor(open).float()
  open_realreal.append(calculate_FID(real_data, real_data, netD))

  # closed_realreal
  real_data = torch.tensor(closed).float()
  closed_realreal.append(calculate_FID(real_data, real_data, netD))

  # noise open
  real_data = torch.tensor(open).float()
  noise_data = torch.randn((42, 64, 3152)).float()
  open_realnoise.append(calculate_FID(real_data, noise_data, netD))

  # noise closed
  real_data = torch.tensor(closed).float()
  noise_data = torch.randn((45, 64, 3152)).float()
  closed_realnoise.append(calculate_FID(real_data, noise_data, netD))

print("open_realfake: ", np.mean(open_realfake))
print("closed_realfake: ", np.mean(closed_realfake))
print("open_realreal: ", np.mean(open_realreal))
print("closed_realreal: ", np.mean(closed_realreal))
print("open_realnoise: ", np.mean(open_realnoise))
print("closed_realnoise: ", np.mean(closed_realnoise))


Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.5529513955116272
Final Test Accuracy: 0.8012152910232544
Final Test Accuracy: 0.7769097089767456
Final Test Accuracy: 0.7248263955116272
Final Test Accuracy: 0.8315972089767456
Final Test Accuracy: 0.8307291865348816
Final Test Accuracy: 0.7222222089767456
Final Test Accuracy: 0.8888888955116272
Final Test Accuracy: 0.7222222089767456
Final Test Accuracy: 0.7395833134651184
Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.7222222089767456
Final Test Accuracy: 0.7890625
Final Test Accuracy: 0.835069477558136
Final Test Accuracy: 0.7222222089767456
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8888888955116272
Final Test Accuracy: 0.8298611044883728
Final Test Accuracy: 0.7196180820465088
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8810763955116272
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.734375
Final Test Accuracy:

In [12]:
pre_accuracy = [0.7777777910232544, 0.5529513955116272, 0.8012152910232544, 0.7769097089767456, 0.7248263955116272, 0.8315972089767456, 0.8307291865348816, 0.7222222089767456, 0.8888888955116272, 0.7222222089767456, 0.7395833134651184, 0.7777777910232544, 0.7222222089767456, 0.7890625, 0.835069477558136, 0.7222222089767456, 0.8333333134651184, 0.8333333134651184, 0.8888888955116272, 0.8298611044883728, 0.7196180820465088, 0.8333333134651184, 0.8810763955116272, 0.8333333134651184, 0.734375, 0.8333333134651184, 0.7795138955116272, 0.8888888955116272, 0.7925347089767456, 0.7847222089767456, 0.8333333134651184, 0.7795138955116272, 0.7282986044883728, 0.8888888955116272, 0.7222222089767456, 0.8559027910232544, 0.7777777910232544, 0.8333333134651184, 0.8298611044883728, 0.8888888955116272, 0.8333333134651184, 0.7213541865348816, 0.8333333134651184, 0.78125, 0.875, 0.7222222089767456, 0.8333333134651184, 0.780381977558136, 0.8333333134651184, 0.6840277910232544, 0.7630208134651184, 0.8229166865348816, 0.7795138955116272, 0.7777777910232544, 0.7604166865348816, 0.811631977558136, 0.8333333134651184, 0.8333333134651184, 0.7552083134651184, 0.780381977558136, 0.8255208134651184, 0.8888888955116272, 0.7265625, 0.8333333134651184, 0.7786458134651184, 0.8888888955116272, 0.780381977558136, 0.7230902910232544, 0.8072916865348816, 0.7222222089767456, 0.6666666865348816, 0.7560763955116272, 0.8333333134651184, 0.7777777910232544, 0.8342013955116272, 0.7777777910232544, 0.7439236044883728, 0.780381977558136, 0.7170138955116272, 0.6354166865348816, 0.8333333134651184, 0.8810763955116272, 0.6744791865348816, 0.7690972089767456, 0.7222222089767456, 0.8333333134651184, 0.8888888955116272, 0.8888888955116272, 0.8333333134651184, 0.7222222089767456, 0.8220486044883728, 0.7855902910232544, 0.944444477558136, 0.7578125, 0.780381977558136, 0.8333333134651184, 0.8888888955116272, 0.703125, 0.7777777910232544, 0.8237847089767456]
np.mean(pre_accuracy)


0.7929600697755813

# Accuracy

over 100 iterations

In [13]:
open = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-open-64ch.npy")
closed = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-closed-64ch.npy")
open_generated1 = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-open-1.npy")
closed_generated1 = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-closed-1.npy")
open_generated = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-open-1.npy")
closed_generated = np.load("/content/gdrive/My Drive/Research_Paper/generated-data/generated-closed-1.npy")

open = np.concatenate((open, open_generated1))
closed = np.concatenate((closed, closed_generated1))
open = np.concatenate((open, open_generated))
closed = np.concatenate((closed, closed_generated))
test_100_accuracies = []

for i in range(100):
  predicted_labels_synthetic_classifier, true_labels_synthetic_classifier, netD, accuracy = train(open, closed)
  test_100_accuracies.append(accuracy)

print("Avg test accuracy: ", np.mean(test_100_accuracies))



Final Test Accuracy: 0.9610849022865295
Final Test Accuracy: 0.9761202931404114
Final Test Accuracy: 1.0
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 0.9451650977134705
Final Test Accuracy: 0.9646226763725281
Final Test Accuracy: 0.963443398475647
Final Test Accuracy: 0.9516509771347046
Final Test Accuracy: 0.9879127740859985
Final Test Accuracy: 0.9433962106704712
Final Test Accuracy: 0.9814268946647644
Final Test Accuracy: 0.9543042778968811
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 0.9265919923782349
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 1.0
Final Test Accuracy: 0.9439858794212341
Final Test Accuracy: 0.9811320900917053
Final Test Accuracy: 0.9622641801834106
Final Test Accuracy: 0.9504716992378235
Final Test Accuracy: 0.9805424809455872
Final Test Accuracy: 0.9622641801834106
Final Test Accuracy: 0.9761202931404114
Final Test Accuracy: 1.0
Final 

In [3]:
open = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-open-64ch.npy")
closed = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-closed-64ch.npy")
open1 = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-open-64ch.npy")
closed1 = np.load("/content/gdrive/My Drive/Research_Paper/Training/normalized-training-closed-64ch.npy")

test_100_accuracies = []

for i in range(100):
  predicted_labels_synthetic_classifier, true_labels_synthetic_classifier, netD, accuracy = train(open, closed, True)
  test_100_accuracies.append(accuracy)

print("Avg test accuracy: ", np.mean(test_100_accuracies))



Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.8185763955116272
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8880208134651184
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8402777910232544
Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.6892361044883728
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8880208134651184
Final Test Accuracy: 0.7222222089767456
Final Test Accuracy: 0.8246527910232544
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.8524305820465088
Final Test Accuracy: 0.8333333134651184
Final Test Accuracy: 0.7777777910232544
Final Test Accuracy: 0.8307291865348816
Final Test Accuracy: 0.8567708134651184
Final Test Accuracy: 0.7760416865348816
Final Test Accuracy: 0.6666666865348816
Final Test Accuracy: 0.8185763955116272
